# 2D Transient Elastic FE model in Gridap
This Program is to create a two dimensional rectangular FE model that is subjected to displacement boundary condition at one of it's walls. The material used here is **elastic**, and the nature of the simulation is **transient**. The displacement rate is being applied monotonously for a few seconds. The FE model and the simulation results could be viewed in paraview. 

In [12]:
# Libraries
using Gridap
using Gridap.Geometry
using Gridap.Adaptivity
using GridapGmsh
using WriteVTK
using LinearAlgebra

# Model creation

In [13]:
model_length = 5.0
model_depth = 2.0
seed_size = 0.1
function create_domain_and_mesh()
    # For complex geometries, consider using GMSH with GridapGmsh
    # Here we create a simplified rectangular domain
    
    domain = (0.0, model_length, 0.0, model_depth)
    n = Int(round(model_length / seed_size))
    m = Int(round((model_depth) / seed_size))
    
    model = CartesianDiscreteModel(domain, (n, m))
    return model
end

create_domain_and_mesh (generic function with 1 method)

In [14]:
# Create domain and mesh
model = create_domain_and_mesh()

CartesianDiscreteModel()

In [15]:
writevtk(model,"model_Lin_Elasticity")

3-element Vector{Vector{String}}:
 ["model_Lin_Elasticity_0.vtu"]
 ["model_Lin_Elasticity_1.vtu"]
 ["model_Lin_Elasticity_2.vtu"]

![FE model](mesh.png)

# Label boundaries

In [16]:
# Add boundary labels before creating triangulation
labels = get_face_labeling(model)

FaceLabeling:
 0-faces: 1071
 1-faces: 2070
 2-faces: 1000
 tags: 10
 entities: 9

In [17]:
# In order to see the labels, open the vtu files in the paraview and identify the tags, and then 
# put them into broader tag bracket. 
# tag_1 -> left-bottom corner
# tag_2 -> right-bottom corner
# tag_3 -> left-top corner
# tag_4 -> right-top corner
# tag_5 -> bottom
# tag_6 -> top
# tag_7 -> left
# tag_8 -> right
add_tag_from_tags!(labels, "bottom", [5])
add_tag_from_tags!(labels, "left", [7]) 
add_tag_from_tags!(labels, "right", [8])
add_tag_from_tags!(labels, "top", [6])

14-element Vector{String}:
 "tag_1"
 "tag_2"
 "tag_3"
 "tag_4"
 "tag_5"
 "tag_6"
 "tag_7"
 "tag_8"
 "interior"
 "boundary"
 "bottom"
 "left"
 "right"
 "top"

# Triangulation and Measure

To perform integration over the interior of the domain Ω, we first construct a triangulation of the region to serve as an integration mesh. Within each cell of this mesh, a Gaussian quadrature rule is applied for accurate numerical integration. The triangulation, together with the associated **Lebesgue measure**, enables the representation of integrals using a syntax that closely resembles conventional mathematical notation.

In [18]:
Ω = Triangulation(model)
order = 2
dΩ = Measure(Ω, order)

GenericMeasure()

# Create Quadrature

In [19]:
# For scalar fields like temperature, pressure, potential etc, use single value (1 DOF per node). 
# Solution: u(x,y) (a scalar function)
# For vector fileds like displacement in elasticity, velocity in fluids etc, use vector value (2 DOF per node)
# Solution: u(x,y) = [u₁(x,y), u₂(x,y)] ie., each node has x and y components
reffe = ReferenceFE(lagrangian, VectorValue{2,Float64}, 1)  # Reduced to linear for stability

(Lagrangian(), (VectorValue{2, Float64}, 1), Base.Pairs{Symbol, Union{}, Tuple{}, @NamedTuple{}}())

# Define FE Spaces

In [20]:
V = TestFESpace(model, reffe, 
                conformity=:H1, 
                dirichlet_tags=["left", "right"], 
                # dirichlet_masks=[(true,true), # left: fix both components
                                # (false,true)] # right: fix only y-component
              ) 

UnconstrainedFESpace()

### Boundary conditions

In [21]:
g_left_static = x -> VectorValue(0.0, 0.0)  # Both components zero
g_right_static = x -> VectorValue(0.0, 0.0)  # Both components zero

#7 (generic function with 1 method)

In [22]:
U_static = TrialFESpace(V, [g_left_static, g_right_static])

TrialFESpace()

# Constitutive law

The symmetric gradient operator is represented by the function `ε` provided by Gridap (also available as `symmetric_gradient`). 
However, function `σ` representing the stress tensor is not predefined in the library and it has to be defined ad-hoc by the user.

In [23]:
displ_rate = 0.1

const E = 70.0e9
const ν = 0.33
const ρ = 2.5
const λ = (E*ν)/((1+ν)*(1-2*ν))
const μ = E/(2*(1+ν))
σ(ε) = λ*tr(ε)*one(ε) + 2*μ*ε

σ (generic function with 1 method)

# Weak Forms

In [11]:
a_static(u,v) = ∫( ε(v) ⊙ (σ∘ε(u)) )*dΩ  # Elastic stiffness
l_static(v) = 0 # External forces

l_static (generic function with 1 method)

# Solve Static

In [31]:
# Solve static FE problem
op_static = AffineFEOperator(a_static, l_static, U_static, V)
uh_static = solve(op_static)

SingleFieldFEFunction():
 num_cells: 1000
 DomainStyle: ReferenceDomain()
 Triangulation: BodyFittedTriangulation()
 Triangulation id: 3818158238650157448

In [25]:
writevtk(Ω,"results_Lin_Elasticity_Static",cellfields=["uh"=>uh_static,"epsi"=>ε(uh_static),"sigma"=>σ∘ε(uh_static)])
    
println("Static simulation completed successfully!")

Static simulation completed successfully!


![Static_figure](static.png)

# Transient FE spaces

In [27]:
g_left(t) = x -> VectorValue(0.0, 0.0)  # Both components zero
g_right(t) = x -> VectorValue(-displ_rate * t, displ_rate^2 * t)  # Only one component zero

g_right (generic function with 1 method)

In [28]:
U = TransientTrialFESpace(V, [g_left, g_right])

TransientTrialFESpace()

# Weak Forms for transient

In [29]:
# External force (time-dependent)
f(t) = VectorValue(0.0, -ρ*9.81) # Body force = density * gravity

# Transient weak forms
a_transient(t, u, v) = ∫( ε(v) ⊙ (σ∘ε(u)) )dΩ # Stiffness form
m(t, dtu, v) = ∫(ρ * (v ⋅ dtu) )dΩ # Mass form 
l_transient(t, v) = ∫( v ⋅ f(t) )dΩ # Force form

l_transient (generic function with 1 method)

# FE Problem

In [30]:
# FE Operator
op_transient = TransientLinearFEOperator((a_transient, m), l_transient, U, V)

TransientLinearFEOpFromWeakForm()

In [33]:
# Initial condition
uh0 = uh_static # static solution

# Transient solver
ls = LUSolver() # LU decomposition solver
Δt = 0.05 # time step
θ = 0.5
solver = ThetaMethod(ls, Δt, θ)

ThetaMethod()

## A note on theta method

It's a **family of time-stepping schemes** for solving ODEs of the form:

$M*du/dt + A*u = F(t)$

The `ThetaMethod` approximates the time derivative as:

$M*(uⁿ⁺¹ - uⁿ)/Δt + A*(θ*uⁿ⁺¹ + (1-θ)*uⁿ) = θ*Fⁿ⁺¹ + (1-θ)*Fⁿ$

Where $uⁿ$ is solution at time $tⁿ$, $uⁿ⁺¹$ at $tⁿ⁺¹ = tⁿ + Δt$.

For `θ = 0.5`, the scheme solves:

$M*(uⁿ⁺¹ - uⁿ)/Δt + A*(0.5*uⁿ⁺¹ + 0.5*uⁿ) = 0.5*Fⁿ⁺¹ + 0.5*Fⁿ$

Rearrange to solve for $uⁿ⁺¹$:

$[M/Δt + 0.5*A] * uⁿ⁺¹ = [M/Δt - 0.5*A] * uⁿ + 0.5*(Fⁿ⁺¹ + Fⁿ)$

We use `θ = 1/2` which is second order accurate, unconditionally stable, no numerical dumping, and symmetric in time.

## A note on LU solver

LU factorization is done once at the beginning. The matrix $[M/Δt + 0.5*A]$ is constant if $Δt$ is constant, ie., in monotonic strain loading. At each time step: only forward/backward substitution, thus fast.

# Transient Solution

In [34]:
t0 = 0.0
tF = 10
    
println("Solving transient problem...")  
sol_t = solve(solver, op_transient, t0, tF, uh0)

Solving transient problem...


GenericTransientFESolution()

In [35]:
# Solving and storing results
createpvd("subduction_transient_results") do pvd
    step = 0
    for (t, u_t) in sol_t
        println("Time step $step: t = $t")
        if step % 10 == 0  # Save only every 5th step
            pvd[t] = createvtk(Ω, "subduction_transient_$step.vtu", 
                                  cellfields=["u" => u_t, "ε_t" => ε(u_t), "σ_t" => σ∘ε(u_t)])
        end
        step += 1
    end
end

Time step 0: t = 0.05
Time step 1: t = 0.1
Time step 2: t = 0.15000000000000002
Time step 3: t = 0.2
Time step 4: t = 0.25
Time step 5: t = 0.3
Time step 6: t = 0.35
Time step 7: t = 0.39999999999999997
Time step 8: t = 0.44999999999999996
Time step 9: t = 0.49999999999999994
Time step 10: t = 0.5499999999999999
Time step 11: t = 0.6
Time step 12: t = 0.65
Time step 13: t = 0.7000000000000001
Time step 14: t = 0.7500000000000001
Time step 15: t = 0.8000000000000002
Time step 16: t = 0.8500000000000002
Time step 17: t = 0.9000000000000002
Time step 18: t = 0.9500000000000003
Time step 19: t = 1.0000000000000002
Time step 20: t = 1.0500000000000003
Time step 21: t = 1.1000000000000003
Time step 22: t = 1.1500000000000004
Time step 23: t = 1.2000000000000004
Time step 24: t = 1.2500000000000004
Time step 25: t = 1.3000000000000005
Time step 26: t = 1.3500000000000005
Time step 27: t = 1.4000000000000006
Time step 28: t = 1.4500000000000006
Time step 29: t = 1.5000000000000007
Time step 30

21-element Vector{String}:
 "subduction_transient_results.pvd"
 "subduction_transient_0.vtu"
 "subduction_transient_10.vtu"
 "subduction_transient_20.vtu"
 "subduction_transient_30.vtu"
 "subduction_transient_40.vtu"
 "subduction_transient_50.vtu"
 "subduction_transient_60.vtu"
 "subduction_transient_70.vtu"
 "subduction_transient_80.vtu"
 "subduction_transient_90.vtu"
 "subduction_transient_100.vtu"
 "subduction_transient_110.vtu"
 "subduction_transient_120.vtu"
 "subduction_transient_130.vtu"
 "subduction_transient_140.vtu"
 "subduction_transient_150.vtu"
 "subduction_transient_160.vtu"
 "subduction_transient_170.vtu"
 "subduction_transient_180.vtu"
 "subduction_transient_190.vtu"

![transient_at_initial_stages](result_20.png)

![transient_sigma_at_latter_stages](result_190.png)